# Tidybear Verbs

Inspired by dplyr, tidybear uses "verbs" to wrangle pandas dataframes. This notebook shows how to use tidybear to perform actions like:

- Count by group
- Top n rows ordered by some column
- Rename columns
- Transform data from long to wide format (or vice versa)

In [1]:
import pandas as pd
import tidybear as tb

from seaborn import load_dataset

I'll use the planets dataset to explore some tidybear verbs.

In [2]:
data = load_dataset("planets")
data

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


## Counting

Counting is the most fundamental data manipulation. While something like `df.column.value_counts()` is useful for counting values in a single column, tidybear provides a more general way to count values in accross any number of columns. It also always returns a dataframe which is useful for further analysis.

Starting with a single column, let's get the counts of the number of planets found by each method. Notice the default sorting is by the column values.

In [3]:
tb.count(data, "method")

,method,n
0,Astrometry,2
1,Eclipse Timing Variations,9
2,Imaging,38
3,Microlensing,23
4,Orbital Brightness Modulation,3
5,Pulsar Timing,5
6,Pulsation Timing Variations,1
7,Radial Velocity,553
8,Transit,397
9,Transit Timing Variations,4


Next, we can specify `sort=True` to sort the results by the number of planets instead of the column values. This can be combined with `.head()` to get the top results.

In [4]:
tb.count(data, "method", sort=True).head(5)

,method,n
7,Radial Velocity,553
8,Transit,397
2,Imaging,38
3,Microlensing,23
1,Eclipse Timing Variations,9


And finally, we can count the number of planets found in each year + method by providing a list of columns. I've filtered to only years > 2012 to make the dataframe smaller, later we'll use `slice_max` to get the "best" method for each year.

In [5]:
tb.count(data[data.year >= 2012], ["year", "method"])

,year,method,n
0,2012,Eclipse Timing Variations,1
1,2012,Imaging,2
2,2012,Microlensing,6
3,2012,Radial Velocity,38
4,2012,Transit,92
5,2012,Transit Timing Variations,1
6,2013,Astrometry,1
7,2013,Imaging,7
8,2013,Microlensing,4
9,2013,Orbital Brightness Modulation,1


## Slicing

Slicing is a way to get the top n rows of a dataframe. Tidybear povides `slice_max` for the most common rows and `slice_min` for the least common rows. They each allow for a variety of options to be specified, like how many rows, what column to order by, and whether to get the top rows within a group.

Let's find the top 3 planets that are furthest away.

In [6]:
tb.slice_max(data, order_by="distance", n=3)

,method,number,orbital_period,mass,distance,year
951,Transit,1,4.200,NaN,8500.0,2006
952,Transit,1,1.796,NaN,8500.0,2006
911,Microlensing,1,NaN,NaN,7720.0,2012


How about the 3 planets that are the lowest in mass?

In [7]:
tb.slice_min(data, order_by="mass", n=3)

,method,number,orbital_period,mass,distance,year
46,Radial Velocity,1,3.23570,0.00360,1.35,2012
128,Radial Velocity,4,3.14942,0.00600,6.27,2005
263,Radial Velocity,3,40.11400,0.00755,6.06,2011


Finally, lets answer the question of what method found the most planets in each year since 2005.

In [8]:
counts = tb.count(data[data.year >= 2005], ["year", "method"])
tb.slice_max(counts, order_by="n", n=1, groupby="year")

,year,method,n
0,2005,Radial Velocity,36
1,2006,Radial Velocity,21
2,2007,Radial Velocity,35
3,2008,Radial Velocity,43
4,2009,Radial Velocity,74
5,2010,Transit,48
6,2011,Radial Velocity,94
7,2012,Transit,92
8,2013,Transit,75
9,2014,Transit,40


## Pivoting

Pivoting is the process of turning data from a long format into a wide format, or a wide format to a long format. Each format type is useful in it's own way, but it is important to be able to transition from one to the other seamlessly.

Tidybear provides `pivot_longer` and `pivot_wider` verbs to do this. 

Let's start from pivoting from wide to long, so we'll use `pivot_longer`. This is useful when you have multiple variables you want to plot again using a tool like plotnine (a ggplot2 implementation in python). This dataset doesn't have a "planet_id" column, so I will add one, assuming each row is a unique planet.

In [9]:
planets_with_ids = data.reset_index().rename(columns={"index": "planet_id"})

tb.pivot_longer(
    planets_with_ids,
    ["orbital_period", "mass", "distance"],
    names_to="metric",
    cols_are="targets",
).head(9)


,planet_id,method,number,year,metric,value
0,0,Radial Velocity,1,2006,orbital_period,269.300
1,0,Radial Velocity,1,2006,mass,7.100
2,0,Radial Velocity,1,2006,distance,77.400
3,1,Radial Velocity,1,2008,orbital_period,874.774
4,1,Radial Velocity,1,2008,mass,2.210
5,1,Radial Velocity,1,2008,distance,56.950
6,2,Radial Velocity,1,2011,orbital_period,763.000
7,2,Radial Velocity,1,2011,mass,2.600
8,2,Radial Velocity,1,2011,distance,19.840


Because we are pivoting from wide to long, we must specify what columns are going to be transposed or what columns are going to stay locked. In this case, we want the columns "orbital_period", "distance", and "mass" to become the values of our new column "metric", and their values to become the values of the column "values" (this name can be changed with the `values_to` argument).

In the example above, I specified the columns to be transposed, but I could have also specified the columns to be locked, and left `cols_are="index"`.

Now to explore `tb.pivot_wider`. For this example, I'll can compare the number of planets found by each method that are more than 100 light years away in a more visually appealing way. While this might not be the best way to plot the data, it is an easier way to view the table.

In [10]:
counts = tb.count(
    data.assign(over_100_ly=lambda x: x.distance > 100),
    ["over_100_ly", "method"],
)

tb.pivot_wider(counts, names_from="over_100_ly", values_from="n", fill_value=0)


,method,False,True
0,Astrometry,2.0,0.0
1,Eclipse Timing Variations,5.0,4.0
2,Imaging,28.0,10.0
3,Microlensing,13.0,10.0
4,Orbital Brightness Modulation,1.0,2.0
5,Pulsar Timing,4.0,1.0
6,Pulsation Timing Variations,1.0,0.0
7,Radial Velocity,498.0,55.0
8,Transit,190.0,207.0
9,Transit Timing Variations,1.0,3.0


## Renaming

Renaming columns with pandas is a bit of a pain, but tidybear provides a `rename` verb to do this.

**There are 4 ways to rename columns in tidybear**

#### 1. the pandas way: using a dictionary - `{old_name: new_name}`

In [11]:
tb.rename(data, {"mass": "mass_in_jupiters", "distance": "lightyears"}).head()

,method,number,orbital_period,mass_in_jupiters,lightyears,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009



#### 2. the kwarg way: `old_name="new_name"` (a caveat: the old name must be a valid kwarg for this to work)

In [12]:
tb.rename(data, mass="mass_in_jupiters", distance="lightyears").head()

,method,number,orbital_period,mass_in_jupiters,lightyears,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


While the first two ways can be used to rename any number of columns, the last two ways are used to rename all the columns in a dataframe.
#### 3. the list of strings way: `["col1", "col2", "col3"]`

In [13]:
tb.rename(data[["mass", "distance"]], ["mass_in_jupiters", "lightyears"]).head()

,mass_in_jupiters,lightyears
0,7.10,77.40
1,2.21,56.95
2,2.60,19.84
3,19.40,110.62
4,10.50,119.47


#### 4. the args way - `"col1", "col2", "col3"`

In [14]:
tb.rename(data[["mass", "distance"]], "mass_in_jupiters", "lightyears").head()

,mass_in_jupiters,lightyears
0,7.10,77.40
1,2.21,56.95
2,2.60,19.84
3,19.40,110.62
4,10.50,119.47


I'd also like to point out the pyjanitor package, which is a really great data cleaning package (that sadly I didn't know existed until after I had started creating tidybear). Their way to rename columns is very elegant.

In [17]:
import janitor

data.rename_column("mass", "mass_in_jupiters").head()

,method,number,orbital_period,mass_in_jupiters,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009
